In [1]:
import findspark
import os
findspark.init('/opt/spark')
java8_location= '/usr/lib/jvm/java-1.8.0-openjdk-1.8.0.252.b09-2.el8_1.x86_64'

os.environ['JAVA_HOME'] = java8_location

In [2]:
from pyspark import SparkContext, SparkConf
import pyspark
conf = SparkConf().setAppName("Ruihong").set("spark.executor.memory", "2g").set("spark.driver.memory", "2g")
conf = conf.set("spark.worker.timeout", "250000").set("spark.maxRemoteBlockSizeFetchToMem", "5M").set("spark.network.timeout", "300000").set("spark.executor.heartbeatInterval", "200000")
#.set("spark.maxRemoteBlockSizeFetchToMem", "5M")
sc = SparkContext.getOrCreate(conf)
#sc.setLogLevel("ERROR")
#sc =SparkContext(appName="Ruihong")


In [3]:
print(sc._conf.getAll())
import geopandas as gpd
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import *
import urllib
import os
from matplotlib import pyplot as plt
import geopy.distance
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import *
import shapely.speedups
import time
shapely.speedups.enable() # this makes some spatial queries run faster


[('spark.executor.memory', '2g'), ('spark.executor.heartbeatInterval', '200000'), ('spark.executor.id', 'driver'), ('spark.app.name', 'Ruihong'), ('spark.driver.host', 'datahub.geos.tamu.edu'), ('spark.maxRemoteBlockSizeFetchToMem', '5M'), ('spark.app.id', 'local-1589668128460'), ('spark.rdd.compress', 'True'), ('spark.driver.memory', '2g'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.deployMode', 'client'), ('spark.worker.timeout', '250000'), ('spark.driver.port', '43255'), ('spark.network.timeout', '300000'), ('spark.ui.showConsoleProgress', 'true')]


In [4]:
import os
os.chdir("/vol_fast/tti/ruihongPro")

In [5]:
spark = SparkSession.builder.getOrCreate()
fifa_df1 = spark.read.csv("/vol_fast/tti/ruihongPro/road_info_July.csv", header = True).select(['TripId', 'road_att', 'Longitude'])
fifa_df2 = spark.read.csv("/vol_fast/tti/ruihongPro/road_info_June.csv", header = True).select(['TripId', 'road_att', 'Longitude'])
fifa_df3 = spark.read.csv("/vol_fast/tti/ruihongPro/road_info_Oct1.csv", header = True).select(['TripId', 'road_att', 'Longitude'])
fifa_df4 = spark.read.csv("/vol_fast/tti/ruihongPro/road_info_Feb1.csv", header = True).select(['TripId', 'road_att', 'Longitude'])
fifa_df5 = spark.read.csv("/vol_fast/tti/ruihongPro/road_info_Oct2.csv", header = True).select(['TripId', 'road_att', 'Longitude'])
fifa_df6 = spark.read.csv("/vol_fast/tti/ruihongPro/road_info_Feb2.csv", header = True).select(['TripId', 'road_att', 'Longitude'])
fifa_df = fifa_df1.unionAll(fifa_df2)
fifa_df = fifa_df.unionAll(fifa_df3)
fifa_df = fifa_df.unionAll(fifa_df4)
fifa_df = fifa_df.unionAll(fifa_df5)
fifa_df = fifa_df.unionAll(fifa_df6)

In [6]:
fifa_df1.printSchema()

root
 |-- TripId: string (nullable = true)
 |-- road_att: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [7]:
fifa_df.road_att


Column<b'road_att'>

In [8]:
fifa_df2.printSchema()

root
 |-- TripId: string (nullable = true)
 |-- road_att: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [9]:

fifa_df.select('road_att').distinct().show(truncate = False)

+---------------------------------------------------+
|road_att                                           |
+---------------------------------------------------+
|Principal Arterial ? Other Freeways and Expressways|
|None                                               |
|Minor Arterial                                     |
|Principal Arterial ? Other                         |
|Principal Arterial                                 |
|Collector                                          |
|Other Freeway and Expressway                       |
|Minor Collector                                    |
|Local                                              |
|Interstate                                         |
|Major Collector                                    |
+---------------------------------------------------+



In [10]:

cnt_cond = lambda cond: F.sum(F.when(cond, 1).otherwise(0))



In [11]:
# road_info = fifa_df.groupBy("TripId").agg(((cnt_cond(fifa_df.road_att == 'Local'))/F.count('Longitude')).alias('Local'), 
#                                         ((cnt_cond(fifa_df.road_att == 'Collector'))/F.count('Longitude')).alias('Collector'), 
#                                         ((cnt_cond(fifa_df.road_att == 'Principal Arterial'))/F.count('Longitude')).alias('Principal Arterial'), 
#                                         ((cnt_cond(fifa_df.road_att == 'Minor Arterial'))/F.count('Longitude')).alias('Minor Arterial'), 
#                                         ((cnt_cond(fifa_df.road_att == 'Interstate'))/F.count('Longitude')).alias('Interstate'), 
#                                         ((cnt_cond(fifa_df.road_att == 'Other Freeway and Expressway'))/F.count('Longitude')).alias('Other Freeway and Expressway'), 
#                                         ((cnt_cond(fifa_df.road_att == 'Principal Arterial ? Other Freeways and Expres...')).alias('Principal Arterial Other Freeway and Expressway'),
#                                         ((cnt_cond(fifa_df.road_att == 'Principal Arterial ? Other'))/F.count('Longitude')).alias('Principal Arterial Other'), 
#                                         ((cnt_cond(fifa_df.road_att == 'Major Collector'))/F.count('Longitude')).alias('Major Collector'), 
#                                         ((cnt_cond(fifa_df.road_att == 'Minor Collector'))/F.count('Longitude')).alias('Minor Collector'),
#                                         ((cnt_cond(fifa_df.road_att == 'None'))/F.count('Longitude')).alias('None'))
road_info = fifa_df.groupBy("TripId").agg(((cnt_cond(fifa_df.road_att == 'Local'))/F.count('Longitude')).alias('Local'), 
                                        ((cnt_cond((fifa_df.road_att == 'Collector')|(fifa_df.road_att == 'Major Collector')|(fifa_df.road_att == 'Minor Collector')))/F.count('Longitude')).alias('Collector'), 
                                        ((cnt_cond((fifa_df.road_att == 'Principal Arterial')|(fifa_df.road_att == 'Interstate')|(fifa_df.road_att == 'Other Freeway and Expressway')|(fifa_df.road_att == 'Principal Arterial ? Other Freeways and Expressways')|(fifa_df.road_att == 'Principal Arterial ? Other')))/F.count('Longitude')).alias('Principal Arterial'), 
                                        ((cnt_cond(fifa_df.road_att == 'Minor Arterial'))/F.count('Longitude')).alias('Minor Arterial'), 
                                        ((cnt_cond(fifa_df.road_att == 'None'))/F.count('Longitude')).alias('None'))

In [12]:
road_info.show(truncate = False)

+--------------------------------+--------------------+-------------------+-------------------+--------------------+-------------------+
|TripId                          |Local               |Collector          |Principal Arterial |Minor Arterial      |None               |
+--------------------------------+--------------------+-------------------+-------------------+--------------------+-------------------+
|e78f12ec75b1a760a17a0a1912bf7a03|0.3                 |0.1                |0.0                |0.6                 |0.0                |
|224d2119104d9427e18e67fa4e7fed27|0.0                 |0.5                |0.0                |0.5                 |0.0                |
|4912d1b304671c8e793ae2a37db70259|0.25                |0.25               |0.5                |0.0                 |0.0                |
|99db9c64c0ae5afed2d2fd8f701b2c6d|0.045454545454545456|0.0                |0.45454545454545453|0.045454545454545456|0.45454545454545453|
|35ccb7ca5cde83a973b4739f6a843f49|0.0    

In [13]:
road_info.persist(pyspark.StorageLevel.DISK_ONLY)
road_info.write.format("com.databricks.spark.csv").option("header", "true").save("road_info_Trip_level.csv")


In [14]:
Trip_to_merge1 = spark.read.csv("/vol_fast/tti/ruihongPro/TripRecordsReportTrips_Oct1.csv", header = False)
Trip_to_merge2 = spark.read.csv("/vol_fast/tti/ruihongPro/TripRecordsReportTrips_Oct2.csv", header = False)
Trip_to_merge3 = spark.read.csv("/vol_fast/tti/ruihongPro/TripRecordsReportTrips_Feb1.csv", header = False)
Trip_to_merge4 = spark.read.csv("/vol_fast/tti/ruihongPro/TripRecordsReportTrips_Feb2.csv", header = False)
Trip_to_merge5 = spark.read.csv("/vol_fast/tti/ruihongPro/TripRecordsReportTrips_June.csv", header = False)
Trip_to_merge6 = spark.read.csv("/vol_fast/tti/ruihongPro/TripRecordsReportTrips_July.csv", header = False)
Trip_to_merge = Trip_to_merge1.unionAll(Trip_to_merge2)
Trip_to_merge = Trip_to_merge.unionAll(Trip_to_merge3)
Trip_to_merge = Trip_to_merge.unionAll(Trip_to_merge4)
Trip_to_merge = Trip_to_merge.unionAll(Trip_to_merge5)
Trip_to_merge = Trip_to_merge.unionAll(Trip_to_merge6)
Trip_to_merge = Trip_to_merge.withColumnRenamed("_c0", "TripId").withColumnRenamed("_c1", "DeviceId").withColumnRenamed("_c2", "ProviderId").withColumnRenamed("_c3", "Mode").withColumnRenamed("_c4", "StartDate")
Trip_to_merge = Trip_to_merge.withColumnRenamed("_c5", "StartWDay").withColumnRenamed("_c6", "EndDate").withColumnRenamed("_c7", "EndWDay").withColumnRenamed("_c8", "StartLocLat").withColumnRenamed("_c9", "StartLocLon")
Trip_to_merge = Trip_to_merge.withColumnRenamed("_c10", "EndLocLat").withColumnRenamed("_c11", "EndLocLon").withColumnRenamed("_c12", "IsStartHome").withColumnRenamed("_c13", "IsEndHome").withColumnRenamed("_c14", "GeospatialType")
Trip_to_merge = Trip_to_merge.withColumnRenamed("_c15", "ProviderType").withColumnRenamed("_c16", "ProviderDrivingProfile").withColumnRenamed("_c17", "VehicleWeightClass").withColumnRenamed("_c18", "FirstZoneName").withColumnRenamed("_c19", "LastZoneName")
Trip_to_merge = Trip_to_merge.withColumnRenamed("_c20", "MultipleZones").withColumnRenamed("_c21", "MultipleCorridors")
Trip_to_merge = Trip_to_merge.select(['TripId', 'DeviceId'])


In [15]:
# Trip_to_merge.show(truncate = False)

In [16]:
road_info = Trip_to_merge.join(road_info, on=['TripId'], how='right')

In [17]:

result = road_info.groupBy("DeviceId").agg(F.mean('Local').alias('Average Local'),
                                F.stddev('Local').alias('STD Local'),
                                F.mean('Collector').alias('Average Collector'),
                                F.stddev('Collector').alias('STD Collector'),
                                F.mean('Principal Arterial').alias('Average Principal Arterial'),
                                F.stddev('Principal Arterial').alias('STD Principal Arterial'),
                                F.mean('Minor Arterial').alias('Average Minor Arterial'),
                                F.stddev('Minor Arterial').alias('STD Minor Arterial'),
                                F.mean('None').alias('Average None'),
                                F.stddev('None').alias('STD None')
                               )

result.toPandas().to_csv("road_info_Device_level.csv", index = False, header = True)
